In [1]:
# ============================================
# BLOCK 1: Import Libraries & Setup
# ============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# TensorFlow & Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print("=" * 60)
print("CALLBACKS & TENSORBOARD")
print("=" * 60)
print(f"TensorFlow version: {tf.__version__}")
print("\n✅ Libraries imported successfully!")


CALLBACKS & TENSORBOARD
TensorFlow version: 2.15.0

✅ Libraries imported successfully!


In [2]:
# ============================================
# BLOCK 2: Load & Prepare Fashion MNIST
# ============================================

print("\n" + "=" * 60)
print("DATA PREPARATION - FASHION MNIST")
print("=" * 60)

# Load Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Split validation set
X_valid, y_valid = X_train_full[:5000], y_train_full[:5000]
X_train, y_train = X_train_full[5000:], y_train_full[5000:]

# Scale pixel values
X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0

print(f"\n📊 Data Split:")
print(f"  Training: {X_train.shape[0]:,} samples")
print(f"  Validation: {X_valid.shape[0]:,} samples")
print(f"  Test: {X_test.shape[0]:,} samples")

print("\n✅ Data preparation completed!")


DATA PREPARATION - FASHION MNIST

📊 Data Split:
  Training: 55,000 samples
  Validation: 5,000 samples
  Test: 10,000 samples

✅ Data preparation completed!


In [3]:
# ============================================
# BLOCK 3: ModelCheckpoint - Save Best Model
# ============================================

print("\n" + "=" * 60)
print("MODELCHECKPOINT CALLBACK")
print("=" * 60)

print("\n📌 ModelCheckpoint Purpose:")
print("  • Automatically save model during training")
print("  • Save only when model improves")
print("  • Prevent losing best model if training diverges")

# Create directories
os.makedirs("models", exist_ok=True)

# Build model
model_checkpoint = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_checkpoint.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

# Create ModelCheckpoint callback
checkpoint_cb = callbacks.ModelCheckpoint(
    filepath="models/best_model.keras",
    save_best_only=True,  # Only save when val_loss improves
    monitor="val_loss",    # Metric to monitor
    verbose=1              # Print message when saving
)

print("\n⚙️ ModelCheckpoint Settings:")
print(f"  Filepath: models/best_model.keras")
print(f"  Monitor: val_loss")
print(f"  Save best only: True")

# Train with checkpoint
print("\n🚀 Training with ModelCheckpoint...")
history_cp = model_checkpoint.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb],
    verbose=1
)

print("\n✅ Training with ModelCheckpoint completed!")
print(f"✅ Best model saved at: models/best_model.keras")


MODELCHECKPOINT CALLBACK

📌 ModelCheckpoint Purpose:
  • Automatically save model during training
  • Save only when model improves
  • Prevent losing best model if training diverges



⚙️ ModelCheckpoint Settings:
  Filepath: models/best_model.keras
  Monitor: val_loss
  Save best only: True

🚀 Training with ModelCheckpoint...
Epoch 1/10


1691/1719 [============================>.] - ETA: 0s - loss: 0.7194 - accuracy: 0.7641
Epoch 1: val_loss improved from inf to 0.52283, saving model to models\best_model.keras
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7159 - accuracy: 0.7653 - val_loss: 0.5228 - val_accuracy: 0.8218
Epoch 2/10
1698/1719 [============================>.] - ETA: 0s - loss: 0.4886 - accuracy: 0.8287
Epoch 2: val_loss improved from 0.52283 to 0.43724, saving model to models\best_model.keras
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4883 - accuracy: 0.8289 - val_loss: 0.4372 - val_accuracy: 0.8520
Epoch 3/10
1719/1719 [===

In [4]:
# ============================================
# BLOCK 4: EarlyStopping - Prevent Overfitting
# ============================================

print("\n" + "=" * 60)
print("EARLYSTOPPING CALLBACK")
print("=" * 60)

print("\n📌 EarlyStopping Purpose:")
print("  • Stop training when validation metric stops improving")
print("  • Prevent overfitting")
print("  • Save time (no need to train 100 epochs)")

# Build new model
model_early = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_early.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

# Create EarlyStopping callback
early_stopping_cb = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,           # Stop after 5 epochs without improvement
    restore_best_weights=True,  # Restore best weights
    verbose=1
)

print("\n⚙️ EarlyStopping Settings:")
print(f"  Monitor: val_loss")
print(f"  Patience: 5 epochs")
print(f"  Restore best weights: True")

# Train with early stopping
print("\n🚀 Training with EarlyStopping (max 50 epochs)...")
history_es = model_early.fit(
    X_train, y_train,
    epochs=50,  # Set high, but will stop early
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb],
    verbose=1
)

print(f"\n✅ Training stopped at epoch {len(history_es.history['loss'])}")
print("✅ Best weights restored!")


EARLYSTOPPING CALLBACK

📌 EarlyStopping Purpose:
  • Stop training when validation metric stops improving
  • Prevent overfitting
  • Save time (no need to train 100 epochs)

⚙️ EarlyStopping Settings:
  Monitor: val_loss
  Patience: 5 epochs
  Restore best weights: True

🚀 Training with EarlyStopping (max 50 epochs)...
Epoch 1/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7223 - accuracy: 0.7601 - val_loss: 0.5278 - val_accuracy: 0.8198
Epoch 2/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4896 - accuracy: 0.8288 - val_loss: 0.4393 - val_accuracy: 0.8520
Epoch 3/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4430 - accuracy: 0.8443 - val_loss: 0.5397 - val_accuracy: 0.7946
Epoch 4/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4157 - accuracy: 0.8551 - val_loss: 0.3964 - val_accuracy: 0.8662
Epoch 5/50
1719/1719 [==============================] - 3s 2ms/step - loss: 0.3965 - accuracy: 0.8611

In [5]:
# ============================================
# BLOCK 5: Combining Multiple Callbacks
# ============================================

print("\n" + "=" * 60)
print("COMBINING MULTIPLE CALLBACKS")
print("=" * 60)

print("\n📌 Best Practice:")
print("  • Use BOTH ModelCheckpoint + EarlyStopping")
print("  • Checkpoint saves best model")
print("  • EarlyStopping prevents wasted training time")

# Build new model
model_combined = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dropout(0.2),  # Add dropout for regularization
    layers.Dense(100, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(10, activation="softmax")
])

model_combined.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

# Create multiple callbacks
checkpoint_cb = callbacks.ModelCheckpoint(
    "models/best_combined_model.keras",
    save_best_only=True,
    monitor="val_accuracy",  # Monitor accuracy instead
    mode="max",              # Save when accuracy increases
    verbose=1
)

early_stopping_cb = callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    mode="max",
    restore_best_weights=True,
    verbose=1
)

print("\n⚙️ Combined Callbacks:")
print("  1. ModelCheckpoint:")
print("     - Monitor: val_accuracy (max)")
print("     - Save best only: True")
print("\n  2. EarlyStopping:")
print("     - Monitor: val_accuracy (max)")
print("     - Patience: 10 epochs")

# Train with both callbacks
print("\n🚀 Training with combined callbacks...")
history_combined = model_combined.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb],
    verbose=1
)

print(f"\n✅ Training completed!")
print(f"✅ Total epochs: {len(history_combined.history['loss'])}")


COMBINING MULTIPLE CALLBACKS

📌 Best Practice:
  • Use BOTH ModelCheckpoint + EarlyStopping
  • Checkpoint saves best model
  • EarlyStopping prevents wasted training time

⚙️ Combined Callbacks:
  1. ModelCheckpoint:
     - Monitor: val_accuracy (max)
     - Save best only: True

  2. EarlyStopping:
     - Monitor: val_accuracy (max)
     - Patience: 10 epochs

🚀 Training with combined callbacks...
Epoch 1/100
1700/1719 [============================>.] - ETA: 0s - loss: 0.8401 - accuracy: 0.7141
Epoch 1: val_accuracy improved from -inf to 0.82160, saving model to models\best_combined_model.keras
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8376 - accuracy: 0.7150 - val_loss: 0.5224 - val_accuracy: 0.8216
Epoch 2/100
1709/1719 [============================>.] - ETA: 0s - loss: 0.5641 - accuracy: 0.8067
Epoch 2: val_accuracy improved from 0.82160 to 0.84760, saving model to models\best_combined_model.keras
1719/1719 [==============================] - 4s 2ms/step -

In [6]:
# ============================================
# BLOCK 6: Custom Callback
# ============================================

print("\n" + "=" * 60)
print("CUSTOM CALLBACK")
print("=" * 60)

print("\n📌 Custom Callback Use Cases:")
print("  • Print custom messages during training")
print("  • Save extra information (gradients, activations)")
print("  • Implement custom early stopping logic")
print("  • Send notifications (email, Slack)")

# Define custom callback
class PrintEpochResults(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        """Called at the end of each epoch"""
        if (epoch + 1) % 5 == 0:  # Print every 5 epochs
            print(f"\n🔔 Epoch {epoch + 1} Summary:")
            print(f"   Train Loss: {logs['loss']:.4f}")
            print(f"   Train Acc: {logs['accuracy']*100:.2f}%")
            print(f"   Val Loss: {logs['val_loss']:.4f}")
            print(f"   Val Acc: {logs['val_accuracy']*100:.2f}%")

# Build model
model_custom = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_custom.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

# Create custom callback
custom_cb = PrintEpochResults()

print("\n🚀 Training with custom callback...")
history_custom = model_custom.fit(
    X_train, y_train,
    epochs=15,
    validation_data=(X_valid, y_valid),
    callbacks=[custom_cb],
    verbose=0  # Silent, let custom callback handle printing
)

print("\n✅ Custom callback executed!")


CUSTOM CALLBACK

📌 Custom Callback Use Cases:
  • Print custom messages during training
  • Save extra information (gradients, activations)
  • Implement custom early stopping logic
  • Send notifications (email, Slack)

🚀 Training with custom callback...

🔔 Epoch 5 Summary:
   Train Loss: 0.3999
   Train Acc: 86.05%
   Val Loss: 0.3825
   Val Acc: 86.80%

🔔 Epoch 10 Summary:
   Train Loss: 0.3387
   Train Acc: 88.12%
   Val Loss: 0.3557
   Val Acc: 87.16%

🔔 Epoch 15 Summary:
   Train Loss: 0.2999
   Train Acc: 89.26%
   Val Loss: 0.3311
   Val Acc: 87.96%

✅ Custom callback executed!


In [7]:
# ============================================
# BLOCK 7: TensorBoard - Visualization Tool
# ============================================

print("\n" + "=" * 60)
print("TENSORBOARD - TRAINING VISUALIZATION")
print("=" * 60)

print("\n📌 TensorBoard Features:")
print("  • Real-time training metrics visualization")
print("  • Model graph visualization")
print("  • Histogram of weights/gradients")
print("  • Embedding visualization")
print("  • Profile training performance")

# Create log directory with timestamp
import time
log_dir = os.path.join("logs", f"run_{int(time.time())}")
os.makedirs(log_dir, exist_ok=True)

print(f"\n📁 Log directory: {log_dir}")

# Build model
model_tb = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_tb.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

# Create TensorBoard callback
tensorboard_cb = callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,      # Log weight histograms every epoch
    write_graph=True,      # Visualize model graph
    write_images=False,    # Don't save layer outputs as images
    update_freq='epoch',   # Update logs every epoch
    profile_batch='500,520'  # Profile batches 500-520
)

print("\n⚙️ TensorBoard Settings:")
print(f"  Log directory: {log_dir}")
print(f"  Histogram freq: 1 (every epoch)")
print(f"  Write graph: True")
print(f"  Profile batch: 500-520")

# Train with TensorBoard
print("\n🚀 Training with TensorBoard logging...")
history_tb = model_tb.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[tensorboard_cb],
    verbose=1
)

print("\n✅ Training with TensorBoard completed!")
print(f"\n📊 To view TensorBoard:")
print(f"  1. Open terminal/command prompt")
print(f"  2. Run: tensorboard --logdir=logs")
print(f"  3. Open browser: http://localhost:6006")


TENSORBOARD - TRAINING VISUALIZATION

📌 TensorBoard Features:
  • Real-time training metrics visualization
  • Model graph visualization
  • Histogram of weights/gradients
  • Embedding visualization
  • Profile training performance

📁 Log directory: logs\run_1767293354

⚙️ TensorBoard Settings:
  Log directory: logs\run_1767293354
  Histogram freq: 1 (every epoch)
  Write graph: True
  Profile batch: 500-520

🚀 Training with TensorBoard logging...
Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7245 - accuracy: 0.7638 - val_loss: 0.5181 - val_accuracy: 0.8216
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4871 - accuracy: 0.8293 - val_loss: 0.4406 - val_accuracy: 0.8496
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4417 - accuracy: 0.8451 - val_loss: 0.5268 - val_accuracy: 0.8046
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4148 - accuracy: 0.8557 - val_loss: 

In [8]:
# ============================================
# BLOCK 8: Learning Rate Scheduler
# ============================================

print("\n" + "=" * 60)
print("LEARNING RATE SCHEDULER")
print("=" * 60)

print("\n📌 Why Schedule Learning Rate?")
print("  • Start with large LR for fast initial progress")
print("  • Reduce LR later for fine-tuning")
print("  • Helps escape local minima")
print("  • Improves final accuracy")

# Define learning rate schedule function
def scheduler(epoch, lr):
    """Reduce LR by half every 5 epochs"""
    if epoch < 10:
        return 0.01
    elif epoch < 20:
        return 0.005
    else:
        return 0.001

# Create LearningRateScheduler callback
lr_scheduler_cb = callbacks.LearningRateScheduler(scheduler, verbose=1)

# Build model
model_lr = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_lr.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

print("\n⚙️ Learning Rate Schedule:")
print("  Epochs 0-9:   LR = 0.01")
print("  Epochs 10-19: LR = 0.005")
print("  Epochs 20+:   LR = 0.001")

# Train with LR scheduler
print("\n🚀 Training with LR scheduler...")
history_lr = model_lr.fit(
    X_train, y_train,
    epochs=25,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler_cb],
    verbose=1
)

print("\n✅ Training with LR scheduler completed!")


LEARNING RATE SCHEDULER

📌 Why Schedule Learning Rate?
  • Start with large LR for fast initial progress
  • Reduce LR later for fine-tuning
  • Helps escape local minima
  • Improves final accuracy

⚙️ Learning Rate Schedule:
  Epochs 0-9:   LR = 0.01
  Epochs 10-19: LR = 0.005
  Epochs 20+:   LR = 0.001

🚀 Training with LR scheduler...

Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/25
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7195 - accuracy: 0.7624 - val_loss: 0.5233 - val_accuracy: 0.8244 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/25
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4871 - accuracy: 0.8294 - val_loss: 0.4382 - val_accuracy: 0.8442 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/25
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4429 - accuracy: 0.8446 - val_loss: 0.5310 - val_accuracy: 0.8022 - lr: 0.0100

In [9]:
# ============================================
# BLOCK 9: ReduceLROnPlateau - Adaptive LR
# ============================================

print("\n" + "=" * 60)
print("REDUCELRONPLATEAU - ADAPTIVE LEARNING RATE")
print("=" * 60)

print("\n📌 ReduceLROnPlateau:")
print("  • Automatically reduce LR when metric stops improving")
print("  • More adaptive than fixed schedule")
print("  • Monitors validation loss/accuracy")

# Build model
model_plateau = keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(300, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model_plateau.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    metrics=["accuracy"]
)

# Create ReduceLROnPlateau callback
reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,           # Reduce LR by half
    patience=3,           # After 3 epochs without improvement
    min_lr=0.0001,        # Don't go below this
    verbose=1
)

print("\n⚙️ ReduceLROnPlateau Settings:")
print("  Monitor: val_loss")
print("  Factor: 0.5 (reduce by half)")
print("  Patience: 3 epochs")
print("  Min LR: 0.0001")

# Train with ReduceLROnPlateau
print("\n🚀 Training with ReduceLROnPlateau...")
history_plateau = model_plateau.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[reduce_lr_cb],
    verbose=1
)

print("\n✅ Training with ReduceLROnPlateau completed!")


REDUCELRONPLATEAU - ADAPTIVE LEARNING RATE

📌 ReduceLROnPlateau:
  • Automatically reduce LR when metric stops improving
  • More adaptive than fixed schedule
  • Monitors validation loss/accuracy

⚙️ ReduceLROnPlateau Settings:
  Monitor: val_loss
  Factor: 0.5 (reduce by half)
  Patience: 3 epochs
  Min LR: 0.0001

🚀 Training with ReduceLROnPlateau...
Epoch 1/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7187 - accuracy: 0.7635 - val_loss: 0.5105 - val_accuracy: 0.8284 - lr: 0.0100
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4811 - accuracy: 0.8304 - val_loss: 0.4323 - val_accuracy: 0.8522 - lr: 0.0100
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4367 - accuracy: 0.8448 - val_loss: 0.5058 - val_accuracy: 0.8118 - lr: 0.0100
Epoch 4/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4109 - accuracy: 0.8566 - val_loss: 0.3948 - val_accuracy: 0.8640 - lr: 0.0100
Epoch 5/30
1719

In [10]:
# ============================================
# BLOCK 10: Summary - All Callbacks
# ============================================

print("\n" + "=" * 60)
print("CALLBACKS SUMMARY")
print("=" * 60)

summary_data = {
    'Callback': [
        'ModelCheckpoint',
        'EarlyStopping',
        'TensorBoard',
        'LearningRateScheduler',
        'ReduceLROnPlateau',
        'Custom Callback'
    ],
    'Purpose': [
        'Save best model automatically',
        'Stop training when no improvement',
        'Visualize training in real-time',
        'Fixed LR schedule',
        'Adaptive LR reduction',
        'Custom logic during training'
    ],
    'Key Parameters': [
        'save_best_only, monitor',
        'patience, restore_best_weights',
        'log_dir, histogram_freq',
        'scheduler function',
        'factor, patience, min_lr',
        'on_epoch_end, on_batch_end'
    ],
    'Use Case': [
        'Always use in production',
        'Always use to save time',
        'Debug & experiment',
        'Known optimal schedule',
        'Unknown optimal LR',
        'Advanced use cases'
    ]
}

df_callbacks = pd.DataFrame(summary_data)

print("\n" + "=" * 100)
print(df_callbacks.to_string(index=False))
print("=" * 100)

print("\n🎯 BEST PRACTICES:")
print("-" * 60)
print("1. ALWAYS use ModelCheckpoint + EarlyStopping together")
print("2. Use TensorBoard for debugging and experimentation")
print("3. Use ReduceLROnPlateau for adaptive learning rate")
print("4. Create custom callbacks for specific needs")

print("\n💡 PRODUCTION SETUP:")
print("-" * 60)
print("callbacks = [")
print("    ModelCheckpoint('best_model.keras', save_best_only=True),")
print("    EarlyStopping(patience=10, restore_best_weights=True),")
print("    ReduceLROnPlateau(factor=0.5, patience=5),")
print("    TensorBoard(log_dir='logs/')")
print("]")

print("\n✅ Callbacks summary completed!")


CALLBACKS SUMMARY

             Callback                           Purpose                 Key Parameters                 Use Case
      ModelCheckpoint     Save best model automatically        save_best_only, monitor Always use in production
        EarlyStopping Stop training when no improvement patience, restore_best_weights  Always use to save time
          TensorBoard   Visualize training in real-time        log_dir, histogram_freq       Debug & experiment
LearningRateScheduler                 Fixed LR schedule             scheduler function   Known optimal schedule
    ReduceLROnPlateau             Adaptive LR reduction       factor, patience, min_lr       Unknown optimal LR
      Custom Callback      Custom logic during training     on_epoch_end, on_batch_end       Advanced use cases

🎯 BEST PRACTICES:
------------------------------------------------------------
1. ALWAYS use ModelCheckpoint + EarlyStopping together
2. Use TensorBoard for debugging and experimentation
3. Use R